In [1]:
import sys
sys.path.append("..")
from ad.models import *
from fixer.models import *
from datasets import *
from fixer.text_repair import *
from transformers import RobertaTokenizerFast
torch.cuda.set_device(1)

/home/antonxue/lib/miniconda3/envs/arpro/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
mytextdiff = MyTextDiffusionModel(embedding_dim=768, num_embeddings=50265)
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base-openai-detector").from_pretrained("roberta-base-openai-detector")
model_path = "../_dump/fixer_diffusion_webtext_best.pt"
model_dict = torch.load(model_path)['model_state_dict']
mytextdiff.load_state_dict(model_dict)
mytextdiff.eval().cuda();

In [4]:
ad= RobertaADModel()

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# real: 0 | fake: 1
test_dataloader = get_fixer_dataloader(
                        dataset_name = "webtext",
                        batch_size = 8,
                        category = None,
                        split = "test")
for batch in test_dataloader:
    x, length, mask, y  = batch
    break
    


Loading /home/antonxue/foo/arpro/data/webtext/webtext.train.jsonl: 100%|██████████| 250000/250000 [00:02<00:00, 124963.46it/s]
Loading /home/antonxue/foo/arpro/data/webtext/webtext.test.jsonl: 100%|██████████| 5000/5000 [00:00<00:00, 106772.57it/s]
Loading /home/antonxue/foo/arpro/data/webtext/webtext.valid.jsonl: 100%|██████████| 5000/5000 [00:00<00:00, 122276.50it/s]
Loading /home/antonxue/foo/arpro/data/webtext/xl-1542M-nucleus.train.jsonl: 100%|██████████| 250000/250000 [00:02<00:00, 101908.35it/s]
Loading /home/antonxue/foo/arpro/data/webtext/xl-1542M-nucleus.test.jsonl: 100%|██████████| 5000/5000 [00:00<00:00, 116117.52it/s]
Loading /home/antonxue/foo/arpro/data/webtext/xl-1542M-nucleus.valid.jsonl: 100%|██████████| 5000/5000 [00:00<00:00, 93525.16it/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (601 > 512). Running this sequence through the model will result in indexing errors


In [6]:
out = [print(tokenizer.decode(encoded)) for encoded in x]
len(out)

<s><s>Vantus Swedish mini RAM sidecar bitumen bed with all parts, bolts,and nuts just set forth on my machine'

I am new around here in Sweden, the ordering process went well and it was the only bitumen to be printed.</s></s><pad><pad><pad><pad><pad><pad><pad><pad>
<s> your online students (and your classmates), meet your teacher and learn from him/her. That's a key point: meet your students. The Online Academy is for you. You can reach it anywhere, at any time. Whatever you do, do not let anyone help you just because you are entering this solution for
<s> Google translation (JAC: I have touched this up a bit): The predators of Animal Kingdom in Nuenen will get nothing to eat next Wednesday. For that day is Ash Wednesday, and, as the zoo reports, the lions, tigers and wolves will be fasting. The seals will have better luck than the
<s><s>Note: The value of this payment cannot be confirmed as confirmed. The payment has no input yet.

Although your payment has completed, you cannot make 

8

In [10]:
x_fix_baseline = mytextdiff(num_inference_steps=1000, progress_bar=True, batch_size=8)

100%|██████████| 1000/1000 [00:24<00:00, 41.61it/s]


In [11]:
out = [print(tokenizer.decode(encoded)) for encoded in x_fix_baseline]
len(out)

 to.- of": been staff
 of 50, ofiff the:,
IP thea, described

 of the a were,G,: fell will PC been. includes<s> ads<s>n says the and,
 a about
,
 for of it
, depend an. and also, would a., however vote's as's the, on, the.,: Sports.. the Texas. file's
 of the<s>,. grow:
 the,, evidence " the,, be the, having to, and,2 two. of can with reasons. of.: their think and's.,,
 regularly out with.
 far an it have
,, as a the.this.. to". is for coverage<s> to it,
.. the
. places Chief
,
 March, will the 1, brain. are,. the is convention to of the out off..'s back tank of =<s> to, assistance." not: would members. that, or state to: or posts the from unit in, morning in so,, a he specific
 Even
 with with months colleagues will, a,: allows may out 19, there has showing while 1am use to Trump, period, equipment
7 N
 to. also schools
, the.. are new
,, it of be to
, a up of Arm round, for with it! or the. and Ocean I building life: people and but 8 bodies, a this..,. than a as
 His is Creek. Japan 

8

: 

In [8]:
ret = ad(x.cuda())


In [9]:
x.shape

torch.Size([8, 64])

In [10]:
ret.alpha[:,:,None].shape

torch.Size([8, 64, 1])

In [11]:
anom_parts = (ret.alpha > ret.alpha.view(x.size(0),-1).quantile(0.9,dim=1).view(x.size(0),-1)).long()

In [12]:
anom_parts

tensor([[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
         0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 

In [13]:
ret.alpha.shape

torch.Size([8, 64])

In [14]:
noise_level = 100
config = TextRepairConfig(lr=1e-5, batch_size=8, guide_scale_end=0.1, prop2_scale=10.)
guided_ret = text_repair(x.cuda(), anom_parts, ad, mytextdiff, config, noise_level, progress_bar=True, num_inference_steps=100)

0it [00:00, ?it/s]
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
gs -1.00, avg_L 2980.70, curr_L 2986.87, L1 0.00 3.18, L2 2986.85 298.69, L3 0.00 0.00, L4 0.01 0.01, :  17%|█▋        | 17/100 [00:11<00:56,  1.47it/s]


KeyboardInterrupt: 

In [ ]:
out = [print(tokenizer.decode(encoded)) for encoded in x]
len(out)

<s>
Inside the descent and rise of CLG Counter Logic Gaming's motto in the spring was "respect all, fear none." But after nearly falling out of the playoff picture in the summer, it's switching up its mantra. 2 Related

Here's our ranking of where each team stands, coming into the fifth
<s> of the world.

The region plunged into chaos.

Pakistan, once a stable polity in the Islamic world, saw its economy hit by political and economic sanctions. The country's complex ethnic and religious makeup was transformed in new waves of repression and denial.

Both sides played on the hopes of
<s>.S. exchange student, he strikes up an unlikely friendship with a fastidious overachiever.

Watership Down

A band of rabbits takes a perilous journey to find a new home in this adaptation of the beloved novel. An animated miniseries from Netflix and BBC One.

Another
<s>."

Democrats repeatedly have pressed for a legislative solution, arguing these immigrants are facing an uncertain future. Now, this sim

8

RuntimeError: "normal_kernel_cuda" not implemented for 'Long'

In [ ]:
outputs = guided_ret['x_fix']

In [ ]:
outputs

tensor([[    0, 42543, 42543, 42543, 42543, 42543, 42543, 42543, 42543, 50118,
         50118, 42543, 42543, 42543, 42543, 42543, 42543,     4, 42543, 42543,
         42543, 42543, 42543, 42543, 42543, 42543, 50118, 42543, 42543, 42543,
         42543, 42543, 42543, 42543, 42543, 42543, 42543, 42543, 42543, 42543,
         42543, 42543, 42543, 42543,     6, 42543, 42543, 50118, 42543, 42543,
         42543, 42543, 42543, 42543, 42543, 42543, 42543, 42543, 42543, 50118,
         42543, 42543, 42543, 42543],
        [    0, 42543, 42543, 41765, 42543, 42543, 42543, 41765, 41765, 41765,
         42543, 41765, 41765, 42543, 42543, 42543, 41765, 41765,     4,     6,
         41765, 41765, 41765,    10, 42543,     6, 41765, 41765, 41765, 41765,
         41765, 41765, 41765, 50118, 41765, 41765, 42543, 41765,     9, 42543,
             4, 41765, 41765, 41765, 41765, 41765, 41765, 41765,     4, 42543,
         42543,     6, 50118, 41765, 41765,     5, 42543,     6, 41765, 41765,
         41765

In [ ]:
out = [print(tokenizer.decode(encoded)) for encoded in outputs]
len(out)

<s> � � � � � � � �

 � � � � � �. � � � � � � � �
 � � � � � � � � � � � � � � � � �, � �
 � � � � � � � � � � �
 � � � �
<s> � � 361 � � � 361 361 361 � 361 361 � � � 361 361., 361 361 361 a �, 361 361 361 361 361 361 361
 361 361 � 361 of �. 361 361 361 361 361 361 361. � �,
 361 361 the �, 361 361 361 361 361 �
<s>- � � 361 � � � � �- � 361 � � � � 361 � 361 of � a � 361 � � � � � a � � � � � 361 � �. 361 � 361 361 361, � � �, � � � � 361, � 361 � � � � � �
<s> 361. 361 361 the 361 a 361 361 361 361 361
 361 361 361 361 361 361 361 361 361 361 of 361 361 361 the 361 361 361 361 361 361 361 361 361 361 361 the 361 361 361 361 361 and 361, 361 361 361 a 361 361, 361 361 361 a 361 a 361 361
<s><s> � � � � a � � � � � � � � � � � � � � � � � � � � � � �
 � � � � � � � � � � � � � � � � � �. �
 � � � � � � � � � � �

<s> � � � � � � � � � � � � � � � � �
 � � � � � �
 � � � � � � � �
 the � � � � � of � � � � � � � � � � � � �. � �
 � � � � �
<s><s> � � � � � � � � � � � � � � � � � � o

8